In [1]:
from sklearn.svm import SVC
from sklearn.datasets import load_breast_cancer, make_classification
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.metrics import accuracy_score
from sklearn.model_selection import LeaveOneOut

from sampler import sample_from_logreg

from sklearn.gaussian_process.kernels import RBF
from sklearn.metrics.pairwise import rbf_kernel, polynomial_kernel, sigmoid_kernel, laplacian_kernel
import seaborn as sns

import numpy as np
import matplotlib.pyplot as plt

from cv_svm import SVM_smooth
from kernel_svm import SVM_smooth_kernel

import matplotlib
#matplotlib.use("pgf")
#matplotlib.rcParams.update({
#    "pgf.texsystem": "pdflatex",
#    'font.family': 'serif',
#    'text.usetex': True,
#    'pgf.rcfonts': False,
#})

In [2]:
#X, y = load_breast_cancer(return_X_y=True)
#X, y = make_classification(n_samples=100, n_features=50, n_informative=25, class_sep=0.1)
X, _, y = sample_from_logreg(n=100, p=10, seed=120)
n = X.shape[0]
p = X.shape[1]
y[np.where(y == 0)] = -1

In [3]:
X_scaled = StandardScaler().fit_transform(X)
#X_scaled = X
print(X_scaled.shape)

(100, 10)


In [4]:
gamma = 1/p
clf = SVC(kernel='rbf', gamma=gamma, C=0.20)
clf.fit(X_scaled, y)
true_supports = clf.support_
svm_coef = clf.dual_coef_
svm_coef = svm_coef.reshape(-1, 1)
print(svm_coef.shape)
print(true_supports.shape)

(96, 1)
(96,)


In [5]:
y_pred = clf.predict(X_scaled)
accuracy_score(y, y_pred)

0.87

In [6]:
sigma = 1e-1
lbd = 2
#length_scale = 1000
length_scale = 2.75
degree = 6

In [7]:
# calculate maximum bound
from sklearn.metrics import pairwise_distances
m_max = np.min(pairwise_distances(X))**2
print(m_max)
m_max = 0
#gram = RBF(length_scale)(X, X)
#gram = polynomial_kernel(X, X, degree=degree)
gram = sigmoid_kernel(X, X, gamma=.3)
print(f"gram norm is {np.linalg.norm(gram)} | inv norm is {np.linalg.norm(np.linalg.inv(gram))}")
gram_inv = np.linalg.inv(gram)

C = np.linalg.norm(gram.T @ gram)/(n-1)
print(np.linalg.norm(gram_inv) * np.linalg.norm(gram))

# what lambda do we have to pick?
b = np.linalg.norm(gram_inv) * np.linalg.norm(gram) + 10000
big_term = 1/(2 * np.sqrt(1 + (m_max/sigma)**2)**3)

bound = np.linalg.norm(gram_inv) * (lbd * np.linalg.norm(gram) + C * big_term)/lbd
print(f"Bound is {bound}")

lbd_pick = (np.linalg.norm(gram_inv) * C * big_term)/(b - np.linalg.norm(gram_inv) * np.linalg.norm(gram))

lbd_pick

0.0
gram norm is 74.89666571196724 | inv norm is 728.6586656478883
54574.104499158
Bound is 61226.722135598386


1.3305235272880769

In [9]:
l = np.sqrt(1/(2*gamma))
clf = SVM_smooth_kernel(sigma=sigma, lbd=lbd, kernel=sigmoid_kernel, gamma=8)
clf.fit(X_scaled, y, n_iter=1500, eta=0.05/n, approx_cv=True,
        cv=True, log_iacv=True, save_err_approx=True, save_err_cv=True, save_cond_num=True)

y_pred = clf.predict(X_scaled)
accuracy_score(y, y_pred)

| IACV: 0.00009539 | baseline: 0.00011320
| IACV: 0.00018780 | baseline: 0.00022283
| IACV: 0.00027733 | baseline: 0.00032903
| IACV: 0.00036410 | baseline: 0.00043193
| IACV: 0.00044822 | baseline: 0.00053166
| IACV: 0.00052981 | baseline: 0.00062834
| IACV: 0.00060895 | baseline: 0.00072208
| IACV: 0.00068575 | baseline: 0.00081300
| IACV: 0.00076031 | baseline: 0.00090122
| IACV: 0.00083272 | baseline: 0.00098683
| IACV: 0.00090307 | baseline: 0.00106994
| IACV: 0.00097145 | baseline: 0.00115065
| IACV: 0.00103793 | baseline: 0.00122905
| IACV: 0.00110261 | baseline: 0.00130524
| IACV: 0.00116556 | baseline: 0.00137930
| IACV: 0.00122686 | baseline: 0.00145132
| IACV: 0.00128658 | baseline: 0.00152138
| IACV: 0.00134480 | baseline: 0.00158956
| IACV: 0.00140159 | baseline: 0.00165593
| IACV: 0.00145701 | baseline: 0.00172058
| IACV: 0.00151114 | baseline: 0.00178357
| IACV: 0.00156403 | baseline: 0.00184496
| IACV: 0.00161576 | baseline: 0.00190484
| IACV: 0.00166637 | baseline: 0.0

KeyboardInterrupt: 

In [ ]:
coef = clf.u_/np.linalg.norm(clf.u_)
iacv_coef = normalize(clf.approx_cv_obj.iterates, axis=1)
true_coef = normalize(clf.true_cv_obj.iterates, axis=1)
print(np.mean(np.linalg.norm(iacv_coef - true_coef, 2, axis=1)))
print(np.mean(np.linalg.norm(coef - true_coef, 2, axis=1)))

In [ ]:
print(np.std(clf.approx_cv_obj.iterates))
print(np.std(clf.true_cv_obj.iterates))

In [ ]:
print(np.mean(clf.approx_cv_obj.iterates, axis=0))
print(np.mean(clf.true_cv_obj.iterates, axis=0))

In [ ]:
plt.hist(np.mean(clf.approx_cv_obj.iterates, axis=0), alpha=0.5, bins=40, label="IACV")
plt.hist(np.mean(clf.true_cv_obj.iterates, axis=0), alpha=0.5, bins=40, label="true")
plt.legend()

In [ ]:
plt.stem(svm_coef/np.linalg.norm(svm_coef))
plt.show()
plt.stem(coef[true_supports]/np.linalg.norm(coef[true_supports]))
plt.show()

In [ ]:
print(svm_coef.shape)
print(coef[true_supports].shape)
print((svm_coef.ravel() - coef[true_supports]).shape)
plt.stem(svm_coef.ravel()/np.linalg.norm(svm_coef) - coef[true_supports]/np.linalg.norm(coef[true_supports]))
print(np.abs(svm_coef.ravel()/np.linalg.norm(svm_coef) - coef[true_supports]/np.linalg.norm(coef[true_supports])).mean())
print(np.median(np.abs(svm_coef.ravel()/np.linalg.norm(svm_coef) - coef[true_supports]/np.linalg.norm(coef[true_supports]))))
plt.show()

In [ ]:
plt.hist(np.linalg.norm(clf.approx_cv_obj.iterates - clf.true_cv_obj.iterates, axis=1), bins=60, label="diffs")
plt.legend()

In [ ]:
fig, ax = plt.subplots()
ax.plot(clf.err_approx_["IACV"], label="IACV", c='black')
#ax.plot(clf.err_approx_["NS"], label="NS", c='red', linestyle="--")
ax.plot(clf.err_approx_["baseline"], label="Baseline", c='green', linestyle="--")
ax.set_yscale('log')
ax.set_xscale('log')
ax.set_xlabel("Iterations (log)")
ax.set_ylabel("Err Approx (log)")
ax.legend()
ws = 1.45 # 1.75
hs = 1.35 # 1.25
fig.set_size_inches(w=5.73/ws, h=3.5/hs)
plt.show()
#plt.savefig('kernel_svm_convergence_err_approx_cont.pgf', bbox_inches='tight', pad_inches=0)

In [ ]:
fig, ax = plt.subplots()
ax.plot(clf.err_cv_["IACV"], label="IACV", c='black')
ax.plot(clf.err_cv_["baseline"], label="Baseline", c='green', linestyle="--")
ax.set_yscale('log')
ax.set_xscale('log')
ax.set_xlabel("Iterations (log)")
ax.set_ylabel("Err CV (log)")
ax.legend()
ws = 1.45 # 1.75
hs = 1.35 # 1.25
fig.set_size_inches(w=5.73/ws, h=3.5/hs)
plt.show()
#plt.savefig('kernel_svm_convergence_err_cv_cont.pgf', bbox_inches='tight', pad_inches=0)

In [ ]:
fig, ax = plt.subplots()
cond_nums = np.asarray(clf.approx_cv_obj.cond_nums)
ax.plot(np.mean(cond_nums, axis=1), label="Mean Cond no.", c='black')
ax.set_xlabel("Iterations")
ax.set_ylabel("Condition No")
ax.legend()
ws = 1.45 # 1.75
hs = 1.35 # 1.25
fig.set_size_inches(w=5.73/ws, h=3.5/hs)
plt.show()
plt.savefig('kernel_svm_convergence_cond_no_fail.pgf', bbox_inches='tight', pad_inches=0)

## fig, ax = plt.subplots()
start = 0
ax.plot(lambdas[start:], iacv_means[start:], label="IACV")
ax.plot(lambdas[start:], true_means[start:], label="true", linestyle="--")
ax.set_xlabel("$\lambda$")
ax.set_ylabel("Mean of CV Iterates")
ax.legend()
ws = 1.45 # 1.75
hs = 1.35 # 1.25
fig.set_size_inches(w=5.73/ws, h=3.5/hs)
plt.show()
#plt.savefig('svmtest_lambda_coef_mod.pgf', bbox_inches='tight', pad_inches=0)